# Fetching data from API

First, let's inspect the data and POST request format.

In [1]:
import numpy as np 
import pandas as pd 
import requests

data = pd.read_csv("../input/dialect-dataset/dialect_dataset.csv")

In [2]:
data.head()

,id,dialect
0,1175358310087892992,IQ
1,1175416117793349632,IQ
2,1175450108898565888,IQ
3,1175471073770573824,IQ
4,1175496913145217024,IQ


In [3]:
data.groupby('dialect').count()

,id
dialect,
AE,26296
BH,26292
DZ,16183
EG,57636
IQ,15497
JO,27921
KW,42109
LB,27617
LY,36499


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458197 non-null  int64 
 1   dialect  458197 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


The API needs the id to be a string.

In [5]:
data =data.astype({"id": str})

Used yield not for a particular reason, I just wanted to practice more on using it.

In [6]:
Max_length = 458000
url = "https://recruitment.aimtechnologies.co/ai-tasks"
def getting_1000_texts():
    count = 0
    

    while True:
        if count == 458000:
            break
        id_list = data['id'][count:count+1000].to_list()
        dialect_list = data['dialect'][count:count+1000].to_list()
        response = requests.post(url, json =id_list)
        text_list = list(response.json().values())
        
        yield id_list, dialect_list, text_list
        count += 1000
        

In [7]:
ID_list = []
dialect_lis = []
text_lis = []

for ID, dialect, text in getting_1000_texts():
    ID_list.extend(ID)
    dialect_lis.extend(dialect)
    text_lis.extend(text)

### Getting the last 197 tweet

In [8]:
# getting the last 197 tweet
id_list = data['id'][Max_length:].to_list()
dialect_list = data['dialect'][Max_length:].to_list()
response = requests.post(url, json =id_list)
text_list = list(response.json().values())
ID_list.extend(id_list)
dialect_lis.extend(dialect_list)
text_lis.extend(text_list)
len(ID_list)

458197

In [9]:
dict_lists = {'id': ID_list, 'dialect': dialect_lis, 'text': text_lis} 
    
df_text = pd.DataFrame(dict_lists)

In [10]:
df_text.head()

,id,dialect,text
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [11]:
df_text.to_csv("dialect_data.csv",index=False)